In [1]:
import pandas as pd
import os
import numpy as np ## bug
import gpflow
import tensorflow as tf
import datetime as dt
from gpflow.kernels import ChangePoints, Matern32
from typing import Dict, List, Optional, Tuple, Union
from tensorflow_probability import bijectors as tfb
from sklearn.preprocessing import StandardScaler

Kernel = gpflow.kernels.base.Kernel
MAX_ITERATIONS = 50

In [2]:
from vnstock import Vnstock
def VN_Stock_close_data(start_time, end_time, list_choice = 'VN30', interval = '1D'):
    """
    function để fetch data từ vnstock về, vnindex30
    """
    stock = Vnstock().stock(symbol='ACB', source='VCI')
    stock_list = stock.listing.symbols_by_group(list_choice)
    futures = pd.DataFrame()

    for stock_id in stock_list:
        try:
            stock = Vnstock().stock(symbol= stock_id , source='VCI')
            df = stock.quote.history(start= start_time, end= end_time, interval= interval)
            df = df.set_index('time')
            df = pd.DataFrame(df['close'])
            df.columns = [stock_id]
            df.index = df.index.date
            futures = pd.concat([futures,df],axis = 1, join = 'outer').sort_index()
        except:
            continue

    if interval != '1D':
      futures['Date']= pd.to_datetime(futures.index, format='%Y-%m-%d')
    else:
      futures['Date'] = pd.to_datetime(futures.index, format='%Y-%m-%d %H:%M:%S')
    futures.set_index('Date', inplace=True)

    return futures

Phiên bản Vnstock 3.2.6 đã có mặt, vui lòng cập nhật với câu lệnh : `pip install vnstock --upgrade`.
Lịch sử phiên bản: https://vnstocks.com/docs/tai-lieu/lich-su-phien-ban
Phiên bản hiện tại 3.2.3

Phiên bản Vnai 2.0.4 đã có mặt, vui lòng cập nhật với câu lệnh : `pip install vnai --upgrade`.
Lịch sử phiên bản: https://pypi.org/project/vnai/#history
Phiên bản hiện tại 2.0.2

In [3]:
df = VN_Stock_close_data(
    start_time="2017-12-31",
    end_time= "2025-05-25",
)
data = df['ACB']
# data.head()
daily_return = data.pct_change().dropna()
print(f"✅ daily return with null dropped: \n {daily_return}")
print(f"✅ daily return as array: {daily_return.values}")
daily_return_df = daily_return.to_frame(name = "daily_return")
print(f"daily return of ACB as dataframe: \n {daily_return_df}")

✅ daily return with null dropped: 
 Date
2018-01-03   -0.002937
2018-01-04    0.002946
2018-01-05    0.000000
2018-01-08    0.033774
2018-01-09    0.004261
                ...   
2025-05-19   -0.001881
2025-05-20    0.008011
2025-05-21    0.001870
2025-05-22   -0.003733
2025-05-23    0.011710
Name: ACB, Length: 1842, dtype: float64
✅ daily return as array: [-0.00293686  0.00294551  0.         ...  0.00187003 -0.00373308
  0.0117096 ]
daily return of ACB as dataframe: 
             daily_return
Date                    
2018-01-03     -0.002937
2018-01-04      0.002946
2018-01-05      0.000000
2018-01-08      0.033774
2018-01-09      0.004261
...                  ...
2025-05-19     -0.001881
2025-05-20      0.008011
2025-05-21      0.001870
2025-05-22     -0.003733
2025-05-23      0.011710

[1842 rows x 1 columns]


The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.


In [4]:
# import yfinance as yf
# ticker = "AAPL"
# start_date = "2017-12-31"
# end_date   = "2025-05-25"

# # 2. Download dữ liệu OHLC (ở đây chúng ta chỉ cần 'Close')
# df_raw = yf.download(ticker, start=start_date, end=end_date, interval="1d",
#                      multi_level_index=False)

# # df_raw sẽ có index là datetime, và có cột "Close"
# # 3. Tính daily return: (Close_t / Close_{t-1} - 1)
# df_returns = df_raw[["Close"]].pct_change() \
#                        .dropna() \
#                        .rename(columns={"Close": "daily_return"})

# # 4. Đặt lại tên index thành 'Date' (mặc định df_raw.index đã là datetime index)
# print(f"\n {type(df_returns)}")
# df_returns.index.name = "Date"
# print(f"daily return of {ticker}:\n {df_returns}")

In [5]:
## module service
class ChangePointDetection(ChangePoints):
    def __init__(
            self,
            kernels: Tuple[Kernel, Kernel],
            location: float,
            interval: Tuple[float, float],
            steepness: float = 1.0,
            name: Optional[str] = None
    ):
        if location < interval[0] or location > interval[1]:
            raise ValueError(
                "Location {loc} is not in range [{low},{high}]".format(
                    loc=location, low=interval[0], high=interval[1]
                )
            )
        locations = [location]
        super().__init__(
            kernels = kernels, locations = locations, steepness = steepness, name=name
        )

        affine = tfb.Shift(tf.cast(interval[0], tf.float64))(
            tfb.Scale(tf.cast(interval[1] - interval[0], tf.float64))
        )

        self.locations = gpflow.Parameter(
            locations, transform=tfb.Chain([affine, tfb.Sigmoid()]), dtype = tf.float64
        )

        def _sigmoids(self, X: tf.Tensor):
            locations = tf.reshape(self.locations, (1, 1, -1))
            steepness = tf.reshape(self.steepness, (1, 1, -1))
            return tf.sigmoid(steepness * (X[:, :, None] - locations))

## fit matern kernel - là cái baseline
def fit_matern_kernel(
        time_series_data: pd.DataFrame,
        variance: float = 1.0,
        lengthscale: float = 1.0,
        likelihood_variance: float = 1.0
):

    model = gpflow.models.GPR(
        data = (
            time_series_data.loc[:, ["X"]].to_numpy(),
            time_series_data.loc[:, ["Y"]].to_numpy()
        ),
        kernel = Matern32(variance=variance, lengthscales=lengthscale),
        noise_variance=likelihood_variance
    )

    optimizer = gpflow.optimizers.Scipy()
    nlml = optimizer.minimize(
        model.training_loss, model.trainable_variables, options=dict(maxiter=MAX_ITERATIONS)
    ).fun
    parameters = {
        "kM_variance": model.kernel.variance.numpy(),
        "kM_lengthscales": model.kernel.lengthscales.numpy(),
        "kM_likelihood_variance": model.likelihood.variance.numpy()
    }

    return nlml, parameters

## fit cp kernel - là cái improved nhờ add cp vào?
def fit_changepoint_kernel(
        time_series_data: pd.DataFrame,
        k1_variance: float = 1.0,
        k1_lengthscale: float = 1.0,
        k2_variance: float = 1.0,
        k2_lengthscale: float = 1.0,
        kC_likelihood_variance = 1.0,
        kC_changepoint_location = None,
        kC_steepness = 1.0
):
    if not kC_changepoint_location:
        kC_changepoint_location = (
            time_series_data["X"].iloc[0] + time_series_data["X"].iloc[-1]
        ) / 2.0

    model = gpflow.models.GPR(
        data=(
            time_series_data.loc[:, ["X"]].to_numpy(),
            time_series_data.loc[:, ["Y"]].to_numpy()
        ),
        kernel = ChangePointDetection(
            [
                Matern32(variance=k1_variance, lengthscales=k1_lengthscale),
                Matern32(variance=k2_variance, lengthscales=k2_lengthscale)
            ],
            location=kC_changepoint_location,
            interval=(time_series_data["X"].iloc[0], time_series_data["X"].iloc[-1]),
            steepness=kC_steepness
        )
    )
    model.likelihood.variance.assign(kC_likelihood_variance)

    optimizer = gpflow.optimizers.Scipy()
    nlml = optimizer.minimize(
        model.training_loss, model.trainable_variables, options=dict(maxiter=MAX_ITERATIONS)
    ).fun
    changepoint_location = model.kernel.locations[0].numpy()
    parameters = {
        "k1_variance": model.kernel.kernels[0].variance.numpy().flatten()[0],
        "k1_lengthscale": model.kernel.kernels[0].lengthscales.numpy().flatten()[0],
        "k2_variance": model.kernel.kernels[1].variance.numpy().flatten()[0],
        "k2_lengthscale": model.kernel.kernels[1].lengthscales.numpy().flatten()[0],
        "kC_likelihood_variance": model.likelihood.variance.numpy().flatten()[0],
        "kC_changepoint_location": changepoint_location,
        "kC_steepness": model.kernel.steepness.numpy()
    }

    return changepoint_location, nlml, parameters

## tính severity , là cái v_t, và cái cp_location_normalized, là cái \gamma_t
def changepoint_severity(
     kC_nlml: Union[float, List[float]],
     kM_nlml: Union[float, List[float]]
):
    normalized_nlml = kC_nlml - kM_nlml
    return 1 - 1 / (np.mean(np.exp(-normalized_nlml)) + 1)
def changepoint_loc_and_score(
    time_series_data_window: pd.DataFrame,
    kM_variance: float = 1.0,
    kM_lengthscale: float = 1.0,
    kM_likelihood_variance: float = 1.0,
    k1_variance: float = None,
    k1_lengthscale: float = None,
    k2_variance: float = None,
    k2_lengthscale: float = None,
    kC_likelihood_variance: float = None,
    kC_changepoint_location: float = None,
    kC_steepness=1.0
):
    time_series_data = time_series_data_window.copy()
    Y_data = time_series_data[["Y"]].values
    time_series_data[["Y"]] = StandardScaler().fit(Y_data).transform(Y_data)


    if kM_variance == kM_lengthscale == kM_likelihood_variance == 1.0 :
        (kM_nlml, kM_params) = fit_matern_kernel(time_series_data)
    else:
        (kM_nlml, kM_params) = fit_matern_kernel(time_series_data, kM_variance, kM_lengthscale, kM_likelihood_variance)

    is_cp_location_default = (
        (not kC_changepoint_location)
        or kC_changepoint_location < time_series_data["X"].iloc[0]
        or kC_changepoint_location > time_series_data["X"].iloc[-1]
    )

    if is_cp_location_default:
        kC_changepoint_location = (
            time_series_data["X"].iloc[-1] + time_series_data["X"].iloc[0]
        ) / 2.0

    if not k1_variance:
        k1_variance = kM_params["kM_variance"]

    if not k1_lengthscale:
        k1_lengthscale = kM_params["kM_lengthscales"]

    if not k2_variance:
        k2_variance = kM_params["kM_variance"]

    if not k2_lengthscale:
        k2_lengthscale = kM_params["kM_lengthscales"]

    if not kC_likelihood_variance:
        kC_likelihood_variance = kM_params["kM_likelihood_variance"]


    if (k1_variance == k1_lengthscale == k2_variance == k2_lengthscale == kC_likelihood_variance == kC_steepness == 1.0) and is_cp_location_default:
        (changepoint_location, kC_nlml, kC_params) = fit_changepoint_kernel(time_series_data)
    else:
        (changepoint_location, kC_nlml, kC_params) = fit_changepoint_kernel(
            time_series_data,
            k1_variance=k1_variance,
            k1_lengthscale=k1_lengthscale,
            k2_variance=k2_variance,
            k2_lengthscale=k2_lengthscale,
            kC_likelihood_variance=kC_likelihood_variance,
            kC_changepoint_location=kC_changepoint_location,
            kC_steepness=kC_steepness,
        )

    cp_score = changepoint_severity(kC_nlml, kM_nlml)
    cp_loc_normalised = (time_series_data["X"].iloc[-1] - changepoint_location) / (
        time_series_data["X"].iloc[-1] - time_series_data["X"].iloc[0]
    )

    return cp_score, changepoint_location, cp_loc_normalised, kM_params, kC_params

## run thuật toán
def run_CPD(
    time_series_data: pd.DataFrame,
    lookback_window_length: int,
    start_date: dt.datetime = None,
    end_date: dt.datetime = None,
    use_kM_hyp_to_initialize_kC=True
):
    if start_date and end_date:
        first_window = time_series_data.loc[:start_date].iloc[
             -(lookback_window_length + 1) :, :
         ]

        # first_window = time_series_data.loc[:start_date].iloc[
        #    -(lookback_window_length + 1) :
        # ]
        remaining_data = time_series_data.loc[start_date:end_date, :]
        # remaining_data = time_series_data.loc[start_date:end_date]
        if remaining_data.index[0] == start_date:
            remaining_data = remaining_data.iloc[1:]
        else:
            first_window = first_window.iloc[1:]
        time_series_data = pd.concat([first_window, remaining_data]).copy()
    else:
        raise Exception("Pass start and end date.")

    time_series_data["Date"] = time_series_data.index
    time_series_data = time_series_data.reset_index(drop=True)

    results = []
    for window_end in range(lookback_window_length + 1, len(time_series_data)):
        ts_data_window = time_series_data.iloc[
            window_end - (lookback_window_length + 1) : window_end
        ][["Date", "daily_return"]].copy()
        ts_data_window["X"] = ts_data_window.index.astype(float)
        ts_data_window = ts_data_window.rename(columns={"daily_return": "Y"})
        time_index = window_end - 1
        window_date = ts_data_window["Date"].iloc[-1].strftime("%Y-%m-%d")

        if use_kM_hyp_to_initialize_kC:
            cp_score, cp_loc, cp_loc_normalised, _, _ = changepoint_loc_and_score(ts_data_window)
        else:
            cp_score, cp_loc, cp_loc_normalised, _, _ = changepoint_loc_and_score(
                    ts_data_window,
                    k1_lengthscale=1.0,
                    k1_variance=1.0,
                    k2_lengthscale=1.0,
                    k2_variance=1.0,
                    kC_likelihood_variance=1.0,
                )
        # results.append([window_date, time_index, cp_loc, cp_loc_normalised, cp_score])
        results.append([window_date, cp_loc_normalised, cp_score])
    #results_df = pd.DataFrame(results, columns=["date", "t", "cp_location", "cp_location_norm", "cp_score"])
    results_df = pd.DataFrame(results, columns = ['date', 'cp_location_norm', 'cp_score'])
    return results_df

In [6]:
lookback_window_length = 21
start_date = dt.datetime(2017, 12, 31)
end_date = dt.datetime(2025, 5, 25)

result = run_CPD(
    time_series_data=daily_return_df,
    # time_series_data   = df_returns,
    lookback_window_length=lookback_window_length,
    start_date=start_date,
    end_date=end_date,
    use_kM_hyp_to_initialize_kC=True
)

In [7]:
result

,date,cp_location_norm,cp_score
0,2018-02-01,0.535089,0.648772
1,2018-02-02,0.667476,0.650288
2,2018-02-05,0.517382,0.649128
3,2018-02-06,0.653670,0.633856
4,2018-02-07,0.112936,0.988312
...,...,...,...
1815,2025-05-16,0.404285,0.831232
1816,2025-05-19,0.513865,0.860893
1817,2025-05-20,0.620829,0.747728
1818,2025-05-21,0.668478,0.800450
